Articolul original si codul a retelei neuronale Covid-net gasiti aici https://github.com/lindawangg/COVID-Net

COVID set de date

În prezent, setul de date COVID este construit folosind următoarele seturi de date de radiografie ale pieptului:

* https://github.com/ieee8023/covid-chestxray-dataset
* https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2, glob, random
import matplotlib.pyplot as plt #for plotting things
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname)
# Any results you write to the current directory are saved as output.

/kaggle/input
/kaggle/input/chest-xray-pneumonia
/kaggle/input/chest-xray-pneumonia/chest_xray
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/NORMAL
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/PNEUMONIA
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/NORMAL
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val/PNEUMONIA
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/NORMAL
/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test/PNEUMONIA
/kaggle/input/chest-xray-pneumonia/chest_xray/__MACOSX
/kaggle/input/chest-xray-pneumonia/chest_xray/__MACOSX/chest_xray
/kaggle/input/chest-xray-pneumonia/chest_xray/__MACOSX/chest_xray/train
/kaggle/input/chest-xray-pneumonia/chest_xray/

In [2]:
!ls /kaggle/input/covidchestxraydataset/images

01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg
03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C.jpeg
1-s2.0-S0140673620303706-fx1_lrg.jpg
1-s2.0-S0929664620300449-gr2_lrg-a.jpg
1-s2.0-S0929664620300449-gr2_lrg-b.jpg
1-s2.0-S0929664620300449-gr2_lrg-c.jpg
1-s2.0-S0929664620300449-gr2_lrg-d.jpg
1-s2.0-S0929664620300449-gr3_lrg-a.jpg
1-s2.0-S0929664620300449-gr3_lrg-b.jpg
1-s2.0-S0929664620300449-gr3_lrg-c.jpg
1-s2.0-S0929664620300449-gr3_lrg-d.jpg
1-s2.0-S1684118220300608-main.pdf-001.jpg
1-s2.0-S1684118220300608-main.pdf-002.jpg
1-s2.0-S1684118220300682-main.pdf-002-a1.png
1-s2.0-S1684118220300682-main.pdf-002-a2.png
1-s2.0-S1684118220300682-main.pdf-003-b1.png
1-s2.0-S1684118220300682-main.pdf-003-b2.png
1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg
171CB377-62FF-4B76-906C-F3787A01CB2E.jpeg
191F3B3A-2879-4EF3-BE56-EE0D2B5AAEE3.jpeg
1B734A89-A1BF-49A8-A1D3-66FAFA4FAC5D.jpeg
1F6343EE-AFEC-4B7D-97F5-62797EE18767.jpeg
21DDEBFD-7F16-4E3E-8F90-CB1B8EE82828.jpeg
23E99E2E-447C-46E5-8EB2-D35D12473C39.png
254B82FC-8

#Transfer learning
* https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

In [3]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(4,activation='softmax')(x) 

/opt/conda/lib/python3.6/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


17227776/17225924 [==============================] - 1s 0us/step


In [5]:
model=Model(inputs=base_model.input,outputs=preds)
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True


#image preprocesing
* https://keras.io/preprocessing/image/

In [6]:

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
def preprocessI(path):
            img = cv2.imread(path)
            #print(img)
            im = cv2.resize(img,(324,324))
            im = datagen.apply_transform(im,{
                'theta':np.random.randint(0,40)-20,
                'tx':np.random.rand()/4,
                'ty':np.random.rand()/4,
                'flip_horizontal':np.random.rand()>0.5
            })
            im = cv2.resize(im,(224,224))
            return im
def getGenerators(imgs_covid, nr=1, folder='train'):
    foldere = glob.glob('../input/chest-xray-pneumonia/chest_xray/chest_xray/'+folder+'/*') 
    while 1:
        x, y = [], []
        #nr = 1
        for i, clasa in enumerate(foldere):
            imgs = glob.glob(clasa+'/*')
            y_ = np.zeros(4)
            y_[i]=1
            #print(clasa)
            for j in range(nr):
                #path = imgs[j]#
                path = random.choice(imgs)
                #print(path)
                im = preprocessI(path)
                x.append(im)            
                y.append(y_)
                #print(im.shape)
        y_ = np.zeros(4)
        y_[3] = 1 
        for j in range(nr):
                #path = imgs_covid[j]#
                path = random.choice(imgs_covid)
                #print(path)
                im = preprocessI(path)
                x.append(im)
                y.append(y_)
        x = keras.applications.mobilenet.preprocess_input(np.array(x))
        yield x, y
covid_img = glob.glob('/kaggle/input/covidchestxraydataset/images/*')
gn_train = getGenerators(covid_img[:60], nr=400, folder='train')
gn_val = getGenerators(covid_img[60:120], nr=100, folder='val')
gn_test = getGenerators(covid_img[120:], nr=100, folder='test')
x_tr,y_tr = next(gn_train)
print('val')
x_val,y_val = next(gn_val)
print('test')
x_test,y_test = next(gn_test)
#plt.imshow(x_test[-1]), len(covid_img)

val
test


NameError: name 'plt' is not defined

In [7]:
y_tr = np.array(y_tr)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [8]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
hi = model.fit(x_tr,y_tr, epochs=15, validation_data = (x_val, y_val), batch_size=29)
acc = model.evaluate(x_test, y_test)
acc

Train on 1200 samples, validate on 300 samples
Epoch 1/15
1200/1200 [==============================] - 119s 99ms/step - loss: 0.3932 - accuracy: 0.8883 - val_loss: 2.0852 - val_accuracy: 0.5900
Epoch 2/15
1200/1200 [==============================] - 103s 86ms/step - loss: 0.1028 - accuracy: 0.9692 - val_loss: 3.3879 - val_accuracy: 0.4833
Epoch 3/15
1200/1200 [==============================] - 103s 86ms/step - loss: 0.1070 - accuracy: 0.9617 - val_loss: 1.1447 - val_accuracy: 0.7867
Epoch 4/15
1200/1200 [==============================] - 103s 86ms/step - loss: 0.0865 - accuracy: 0.9742 - val_loss: 0.1814 - val_accuracy: 0.9267
Epoch 5/15
1200/1200 [==============================] - 104s 86ms/step - loss: 0.1132 - accuracy: 0.9692 - val_loss: 0.3873 - val_accuracy: 0.9067
Epoch 6/15
1200/1200 [==============================] - 105s 88ms/step - loss: 0.0967 - accuracy: 0.9692 - val_loss: 0.7505 - val_accuracy: 0.6933
Epoch 7/15
1200/1200 [==============================] - 108s 90ms/step 

KeyboardInterrupt: 

In [9]:

y_true = np.argmax(y_test,axis=1)
y_ = model.predict(x_test)
y_pred = np.argmax(y_, axis=1)
confusion_matrix(y_true, y_pred)

array([[22, 76,  2],
       [ 1, 98,  1],
       [ 5,  0, 95]])

In [10]:
y_true, y_pred

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [11]:

target_names = ['normal', 'pneumonia', 'covid']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

      normal       0.79      0.22      0.34       100
   pneumonia       0.56      0.98      0.72       100
       covid       0.97      0.95      0.96       100

    accuracy                           0.72       300
   macro avg       0.77      0.72      0.67       300
weighted avg       0.77      0.72      0.67       300



In [12]:
model.save('model.h5')

https://www.tensorflow.org/js/guide/conversion

In [13]:
!pip install tensorflowjs

     |████████████████████████████████| 57 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 104.6 MB 55 kB/s s eta 0:00:01
     |████████████████████████████████| 248 kB 49.7 MB/s eta 0:00:01
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-py3-none-any.whl size=32849 sha256=322cbd6c78853861aebe17b46ab9f35e6eacba5e8861859fcd837eda29d08964
  Stored in directory: /root/.cache/pip/wheels/79/8e/a2/5043c19fe89c93e2ae1adb66d066ff743057e70dd0ccbed62c
Successfully built PyInquirer
ERROR: jupyter-console 6.1.0 has requirement prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
ERROR: ipython 7.12.0 has requirement prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.14 which is incompatible.
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.3
    Uninstalling prompt-toolkit-3.0.3:
      Successfully uninstalled prompt-toolkit-3.0.3


In [14]:
!tensorflowjs_converter --input_format=keras model.h5 tfjs_model

In [15]:
!ls

__notebook_source__.ipynb  model.h5  tfjs_model


In [17]:
!tar -czvf model.tar.gz tfjs_model

tfjs_model/
tfjs_model/group1-shard2of6.bin
tfjs_model/group1-shard3of6.bin
tfjs_model/group1-shard5of6.bin
tfjs_model/group1-shard6of6.bin
tfjs_model/group1-shard1of6.bin
tfjs_model/group1-shard4of6.bin
tfjs_model/model.json
